In [1]:
#https://habr.com/ru/post/689520/
####https://newtechaudit.ru/parser-novostnyh-rss-lent/

In [2]:
import feedparser
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup
import pprint as pp


In [3]:
newsurls = {'Kommersant': 'https://www.kommersant.ru/RSS/news.xml',
'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss',
           'news.ru':'https://news.ru/rss/',
           'finam1':'https://www.finam.ru/analysis/conews/rsspoint/',
           'finam_world':'https://www.finam.ru/international/advanced/rsspoint/'} 

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'ДолЛАР|РубЛ|ЕвРО' #пример таргетов
vector2 = 'ЦБ|СбЕРбАНК|курс'

In [4]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse( rss_url )  
    
def getHeadlines( rss_url ): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions( rss_url ): #функция для получения описания новости
    descriptions = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        descriptions.append(BeautifulSoup(newsitem['description']).get_text())
    return descriptions

def getLinks( rss_url ): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates( rss_url ): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [5]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend( getHeadlines( url ) )
    
for key,url in newsurls.items():
    alldescriptions.extend( getDescriptions( url ) )
    
for key,url in newsurls.items():
    alllinks.extend( getLinks( url ) )
    
for key,url in newsurls.items():
    alldates.extend( getDates( url ) )

d:\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [7]:
df = write_all_news(f_all_news) #все новости


In [8]:
df

,Title,Description,Links,Publication Date
0,МЭА: доходы России от экспорта нефти и газа в ...,В январе 2023 года доходы России от экспорта н...,https://www.kommersant.ru/doc/5844005,"Wed, 01 Mar 2023 08:32:50 +0300"
1,"Лукашенко: будет сильным Китай, будет развиват...",Президент Белоруссии Александр Лукашенко встре...,https://www.kommersant.ru/doc/5843998,"Wed, 01 Mar 2023 08:25:27 +0300"
2,В Госдуму внесены поправки о 15 годах лишения ...,"Спикер Госдумы Вячеслав Володин сообщил, что в...",https://www.kommersant.ru/doc/5844001,"Wed, 01 Mar 2023 08:18:51 +0300"
3,Число погибших при столкновении поездов в Грец...,В результате столкновения пассажирского и груз...,https://www.kommersant.ru/doc/5843994,"Wed, 01 Mar 2023 07:59:04 +0300"
4,В Благовещенске введен режим повышенной готовн...,В нескольких районах Благовещенска на поверхно...,https://www.kommersant.ru/doc/5843985,"Wed, 01 Mar 2023 07:23:04 +0300"
...,...,...,...,...
920,Цены на медь стабильны на фоне слабеющего доллара,"\r\nКотировки меди незначительно снижаются, до...",https://www.finam.ru/publications/item/tseny-n...,"Tue, 14 Feb 2023 08:52:31 +0300"
921,"ВВП Японии в 4 квартале вырос на 0,6% в пересч...","\r\nМОСКВА, 14 фев - РИА Новости/Прайм. ВВП Яп...",https://www.finam.ru/publications/item/vvp-yap...,"Tue, 14 Feb 2023 08:24:28 +0300"
922,Meta в марте планирует новый раунд увольнений ...,\r\nMeta (запрещена в РФ) планирует дальнейшие...,https://www.finam.ru/publications/item/meta-v-...,"Mon, 13 Feb 2023 22:22:31 +0300"
923,Число жертв разрушительного землетрясения в Ту...,\r\nБолее 36 000 погибших из-за землетрясения ...,https://www.finam.ru/publications/item/chislo-...,"Mon, 13 Feb 2023 22:13:25 +0300"


In [9]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
140,Доллар упал ниже 74 руб./$,Курс доллара в ходе торгов на Московской бирже...,https://www.kommersant.ru/doc/5843228,"Tue, 28 Feb 2023 08:38:00 +0300"
534,Российский рынок продолжил «зеленеть» в отсутс...,\r\nРоссийский фондовый рынок провел последнюю...,https://www.finam.ru/publications/item/rossiys...,"Tue, 28 Feb 2023 21:01:29 +0300"
545,Минфин и ЦБ РФ прорабатывают проект указа об о...,\r\nМинфин России и ЦБ РФ прорабатывают проект...,https://www.finam.ru/publications/item/minfin-...,"Tue, 28 Feb 2023 17:38:01 +0300"
549,Глава ВТБ озвучил 7 ключевых направлений разви...,\r\nСложнейший для российской экономики 2022 г...,https://www.finam.ru/publications/item/glava-v...,"Tue, 28 Feb 2023 16:48:15 +0300"
554,Темпы роста ВВП России в этом году будут близк...,"\r\nМОСКВА, 28 фев - РИА Новости/Прайм. Темпы ...",https://www.finam.ru/publications/item/tempy-r...,"Tue, 28 Feb 2023 15:57:57 +0300"
573,"Банк России продал юани на 8,9 млрд рублей с р...",\r\nБанк России продал юани на внутреннем рынк...,https://www.finam.ru/publications/item/bank-ro...,"Tue, 28 Feb 2023 10:50:35 +0300"
574,Остатки средств Банка России на корсчетах выро...,\r\nОстатки средств кредитных организаций в Ба...,https://www.finam.ru/publications/item/ostatki...,"Tue, 28 Feb 2023 10:48:09 +0300"
590,Глава минфина США заявила о правовых препятств...,"\r\nМОСКВА, 27 фев - РИА Новости/Прайм. Минист...",https://www.finam.ru/publications/item/glava-m...,"Mon, 27 Feb 2023 20:30:46 +0300"
602,Мосбиржа расширила возможности на рынке станда...,\r\nС 27 февраля 2023 года Московская биржа пр...,https://www.finam.ru/publications/item/mosbirz...,"Mon, 27 Feb 2023 16:16:52 +0300"
608,Российский рынок выкупил пятничную просадку,"\r\nРоссийский фондовый рынок в понедельник, 2...",https://www.finam.ru/publications/item/rossiys...,"Mon, 27 Feb 2023 14:27:13 +0300"


In [10]:
from natasha import NamesExtractor, MorphVocab, Doc, Segmenter, PER, NewsNERTagger, NewsEmbedding

def extract_names(df_desc):
    emb = NewsEmbedding()
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    ner_tagger = NewsNERTagger(emb)
    names_extractor = NamesExtractor(morph_vocab)
    res = []
    
    for desc in df_desc:
        doc = Doc(desc)
        doc.segment(segmenter)
        doc.tag_ner(ner_tagger)
        for span in doc.spans:
            span.normalize(morph_vocab)
        for span in doc.spans:
            if span.type == PER:
                span.extract_fact(names_extractor)
        res.append([_.normal for _ in doc.spans if _.fact])
    return res

df['Names']=extract_names(df['Description'])
df

,Title,Description,Links,Publication Date,Names
0,МЭА: доходы России от экспорта нефти и газа в ...,В январе 2023 года доходы России от экспорта н...,https://www.kommersant.ru/doc/5844005,"Wed, 01 Mar 2023 08:32:50 +0300",[]
1,"Лукашенко: будет сильным Китай, будет развиват...",Президент Белоруссии Александр Лукашенко встре...,https://www.kommersant.ru/doc/5843998,"Wed, 01 Mar 2023 08:25:27 +0300","[Александр Лукашенко, Ли Кэцяном, Лукашенко]"
2,В Госдуму внесены поправки о 15 годах лишения ...,"Спикер Госдумы Вячеслав Володин сообщил, что в...",https://www.kommersant.ru/doc/5844001,"Wed, 01 Mar 2023 08:18:51 +0300",[Вячеслав Володин]
3,Число погибших при столкновении поездов в Грец...,В результате столкновения пассажирского и груз...,https://www.kommersant.ru/doc/5843994,"Wed, 01 Mar 2023 07:59:04 +0300",[]
4,В Благовещенске введен режим повышенной готовн...,В нескольких районах Благовещенска на поверхно...,https://www.kommersant.ru/doc/5843985,"Wed, 01 Mar 2023 07:23:04 +0300",[]
...,...,...,...,...,...
920,Цены на медь стабильны на фоне слабеющего доллара,"\r\nКотировки меди незначительно снижаются, до...",https://www.finam.ru/publications/item/tseny-n...,"Tue, 14 Feb 2023 08:52:31 +0300",[]
921,"ВВП Японии в 4 квартале вырос на 0,6% в пересч...","\r\nМОСКВА, 14 фев - РИА Новости/Прайм. ВВП Яп...",https://www.finam.ru/publications/item/vvp-yap...,"Tue, 14 Feb 2023 08:24:28 +0300",[]
922,Meta в марте планирует новый раунд увольнений ...,\r\nMeta (запрещена в РФ) планирует дальнейшие...,https://www.finam.ru/publications/item/meta-v-...,"Mon, 13 Feb 2023 22:22:31 +0300",[]
923,Число жертв разрушительного землетрясения в Ту...,\r\nБолее 36 000 погибших из-за землетрясения ...,https://www.finam.ru/publications/item/chislo-...,"Mon, 13 Feb 2023 22:13:25 +0300",[]


In [11]:
import requests 
from bs4 import BeautifulSoup

url = 'https://ria.ru' 
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

news_list = [] 
for item in soup.find_all('a', {'class': 'cell-news__link'}): 
    news_list.append(item.text)

print(news_list)
    
news_names = [news.split('»')[0] for news in news_list] 
print(news_names)

[]
[]
